### Model Ensemble For Final Output

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
from sklearn import cross_validation
print("imported libraries . . . ")

imported libraries . . . 


In [2]:
trainPath = "~/Final/FinalTrainingSet.csv"
testPath = "~/Final/FinalTestSet.csv"
train = pd.read_csv(trainPath)
test = pd.read_csv(testPath)
print("size of train", train.shape)
print("size of train", test.shape)
train.columns.values[2:]

('size of train', (495745, 56))
('size of train', (41597, 56))


array(['user_location_country', 'hotel_cluster', 'user_location_region',
       'user_location_city', 'hotel_country', 'hotel_market',
       'srch_destination_id', 'date_time', 'site_name', 'posa_continent',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt',
       'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_type_id',
       'is_booking', 'cnt', 'hotel_continent', 'year', 'month', 'day',
       'hour', 'part_of_day', 'type_of_day', 'season', 'is_alone',
       'dest_feature_pc1', 'dest_feature_pc2', 'dest_feature_pc3',
       'clickRate', 'bookRate', 'clickMRate', 'bookMRate', 'clickHCRate',
       'bookHCRate', 'clickURate', 'bookURate', 'clickUCRRate',
       'bookUCRRate', 'clickUCRate', 'bookUCRate', 'popScoreHmc_Count',
       'popScoreU_Count', 'popScoreSite_Count', 'popScoreDest_Count',
       'popScoreCityDest_Count', 'popScoreCityH_Count', 'duration_of_stay'], dtype=object)

In [3]:
train.shape

(495745, 56)

In [5]:
495745 * 0.60

297447.0

In [4]:
test.shape

(41597, 56)

In [6]:
### We further downsample data as we are applying ensebling

In [7]:
train_sample = train.sample(frac=0.6, replace=False)

In [8]:
train_sample.shape

(297447, 56)

In [9]:
train = train_sample

#### Categorical data transformation

In [10]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
var_mod = ['user_location_region',  'hotel_continent', 'hotel_country', 'hotel_market', 'srch_destination_type_id']
var_mod1 = ['type_of_day',  'season', 'is_alone', 'type_of_day', 'is_package', 'part_of_day']
le = LabelEncoder()
for i in var_mod:
    train[i] = le.fit_transform(train[i])
    test[i] = le.fit_transform(test[i])
#ohe = OneHotEncoder()    
for name in var_mod1:
    train[name]=le.fit_transform(train[name])
    test[name]=le.fit_transform(test[name])

Model ensemble to predict the final outcome!

# NEXT

### We first start with Random forest Models:

In [11]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [12]:
rf1 = RandomForestClassifier(n_estimators=50, min_weight_fraction_leaf=0.1, random_state=3)
popularityVar = ['popScoreHmc_Count', 'popScoreU_Count', 'popScoreSite_Count',
       'popScoreDest_Count', 'popScoreCityDest_Count',
       'popScoreCityH_Count'] 
rf1.fit(train[popularityVar], train['hotel_cluster'])
test_score = rf1.score(test[popularityVar], test['hotel_cluster'])
print("accuracy score for test final Random Forest M1:",test_score*100)

('accuracy score for test final Random Forest M1:', 7.9428804961896287)


In [13]:
erf = ExtraTreesClassifier(n_estimators=70, min_weight_fraction_leaf=0.1, random_state=3)
popularityVar = ['popScoreHmc_Count', 'popScoreU_Count', 'popScoreSite_Count',
       'popScoreDest_Count', 'popScoreCityDest_Count',
       'popScoreCityH_Count'] 
erf.fit(train[popularityVar], train['hotel_cluster'])
test_score = erf.score(test[popularityVar], test['hotel_cluster'])
print("accuracy score for Extreame Randomized trees:",test_score*100)

('accuracy score for Extreame Randomized trees:', 9.8805202298242669)


In [15]:
rf2 = RandomForestClassifier(n_estimators=50, min_weight_fraction_leaf=0.1, random_state=3)

fatureEng1 = [    'part_of_day', 'type_of_day', 'season', 
                  'duration_of_stay', 'is_alone', 
                  'dest_feature_pc1', 'dest_feature_pc2', 'dest_feature_pc3' ]
rf2.fit(train[fatureEng1], train['hotel_cluster'])
test_score = rf2.score(test[fatureEng1], test['hotel_cluster'])
print("final Random Forest M2 is:",test_score*100)


erf2 = ExtraTreesClassifier(n_estimators=70, min_weight_fraction_leaf=0.1, random_state=3)

fatureEng1 = [    'part_of_day', 'type_of_day', 'season', 
                  'duration_of_stay', 'is_alone', 
                  'dest_feature_pc1', 'dest_feature_pc2', 'dest_feature_pc3' ]
erf2.fit(train[fatureEng1], train['hotel_cluster'])
test_score = erf2.score(test[fatureEng1], test['hotel_cluster'])
print("accuracy score for taccuracy score for Extreame Randomized trees:",test_score*100)

('final Random Forest M2 is:', 6.5221049594922711)
('accuracy score for taccuracy score for Extreame Randomized trees:', 8.8948722263624784)


### Based on thier result we will use ensebling technique

In [ ]:
from sklearn.ensemble import VotingClassifier
rf1 = RandomForestClassifier(n_estimators=50, min_weight_fraction_leaf=0.1, random_state=3)
erf = ExtraTreesClassifier(n_estimators=70, min_weight_fraction_leaf=0.1, random_state=3)
ensebleRF = VotingClassifier(estimators=[('rf1', rf1), ('erf', erf)], voting='soft')
ensebleRF.fit(train[popularityVar], train['hotel_cluster'])

In [22]:
ensembleRF_Score = ensebleRF.score(test[popularityVar],test['hotel_cluster'] )
print("Ensembling Accuaracy Random Forest and Extream tree Classifiers: ",  ensembleRF_Score) 
prediction_m1 = ensebleRF.predict(test[popularityVar])

('Ensembling Accuaracy Random Forest and Extream tree Classifiers: ', 0.10354112075390053)


In [24]:
ensembleRF_Score*100

10.354112075390052

In [46]:
from sklearn.ensemble import VotingClassifier
rf2 = RandomForestClassifier(n_estimators=50, min_weight_fraction_leaf=0.1, random_state=3)
erf2 = ExtraTreesClassifier(n_estimators=70, min_weight_fraction_leaf=0.1, random_state=3)
ensebleRF = VotingClassifier(estimators=[('rf2', rf2), ('erf2', erf2)], voting='soft')
ensebleRF.fit(train[fatureEng1], train['hotel_cluster'])
ensembleRF_Score = ensebleRF.score(test[fatureEng1],test['hotel_cluster'] )
print("Ensembling Accuaracy Random Forest and Extream tree Classifiers: ",  ensembleRF_Score) 
prediction_m2 = ensebleRF.predict(test[fatureEng1])

('Ensembling Accuaracy Random Forest and Extream tree Classifiers: ', 0.093588479938457106)


# NEXT 

#### KNN Model ensemble:

In [25]:
Predset_var = ['popScoreHmc_Count', 'popScoreU_Count', 'popScoreSite_Count',
       'popScoreDest_Count', 'popScoreCityDest_Count',
       'popScoreCityH_Count']

In [26]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=15)

knn.fit(train[Predset_var], train['hotel_cluster']) 
testScore = knn.score(test[Predset_var], test['hotel_cluster'])
print("Accuracy in % is: using KNN model with features using book clicks rate is:", testScore*100)

('Accuracy in % is: using KNN model with features using book clicks rate is:', 7.0798374882804049)


#### We will conisider bagging to aggregate result for knn

In [27]:
#TODO:
from sklearn.ensemble import BaggingClassifier
knn = KNeighborsClassifier(n_neighbors=15)
baggedKnn = BaggingClassifier(base_estimator = knn,  n_estimators=30, random_state = 3 )
baggedKnn.fit(train[Predset_var], train['hotel_cluster']) 
bagggedknntestScore = baggedKnn.score(test[Predset_var], test['hotel_cluster'])
print("Accuracy in % is: bagged KNN is:", bagggedknntestScore*100)
prediction_m5 = baggedKnn.predict(test[popularityVar])

('Accuracy in % is: bagged KNN is:', 7.3803399283602191)


In [49]:
prediction_m5 = baggedKnn.predict(test[popularityVar])

# NEXT

### Naive Bayes:

In [ ]:
final_set = [ 'user_location_region', 'season', 'hotel_continent', 'hotel_country','hotel_market', 
              'type_of_day', 'is_alone','dest_feature_pc1',
              'dest_feature_pc2','dest_feature_pc3','duration_of_stay', 'orig_destination_distance']

from sklearn.naive_bayes import GaussianNB
from sklearn import cross_validation
nb = GaussianNB()
nb.fit(train[final_set], train['hotel_cluster'])

In [29]:
nb_Score = nb.score(test[final_set], test['hotel_cluster'])
print("Naive Bayes Test score:", nb_Score*100)

('Naive Bayes Test score:', 6.9452123951246492)


In [ ]:
final_set = [ 'user_location_region', 'season', 'hotel_continent', 'hotel_country','hotel_market', 
              'type_of_day', 'is_alone','dest_feature_pc1',
              'dest_feature_pc2','dest_feature_pc3','duration_of_stay', 'orig_destination_distance']

from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier()
ab.fit(train[final_set], train['hotel_cluster'])

In [31]:
ab_score = ab.score(test[final_set], test['hotel_cluster'])
print("Adaboost Test score:", ab_score*100)

('Adaboost Test score:', 4.298386902901651)


In [ ]:
#TODO: enseble using majority voting NB + AB

from sklearn.ensemble import VotingClassifier
nb = GaussianNB()
ab = AdaBoostClassifier()
ensebleNb_ab = VotingClassifier(estimators=[('nb', nb), ('ab', ab)], voting='soft')
ensebleNb_ab.fit(train[final_set], train['hotel_cluster'])


In [40]:
ensebleNb_ab_Score = ensebleNb_ab.score(test[final_set],test['hotel_cluster'])
print("Naive Bayes and Adaboosting: ", ensebleNb_ab_Score) 
prediction_m3 = ensebleNb_ab.predict(test[final_set])

('Naive Bayes and Adaboosting: ', 0.069524244536865634)


In [41]:
ensebleNb_ab_Score*100

6.9524244536865636

# NEXT 

### Binary logistic

In [ ]:
final_set = [ 'user_location_region', 'season', 'hotel_continent', 'hotel_country','hotel_market', 
              'type_of_day', 'is_alone','dest_feature_pc1',
              'dest_feature_pc2','dest_feature_pc3','duration_of_stay', 'orig_destination_distance']

from sklearn.linear_model import LogisticRegression
Blr = LogisticRegression(penalty = 'l2', max_iter = 100, random_state = 3, multi_class = 'ovr',
                               n_jobs=-1)
Blr.fit(train[final_set], train['hotel_cluster'])
Blr_score = Blr.score(test[final_set], test['hotel_cluster'])


In [44]:
print("Binary Logistic Regression Test score:", Blr_score*100)
prediction_m4 = Blr.predict(test[final_set])

('Binary Logistic Regression Test score:', 8.8179436017020461)


# NEXT

### Gradient Boosting

In [64]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation
gb = GradientBoostingClassifier(learning_rate = 0.1, n_estimators = 10, max_depth = 3, min_samples_split = 3, 
                                       random_state=3)
#gb_score = cross_validation.cross_val_score(gb, train[final_set], train['hotel_cluster'], cv=5)

#print("CV Score of GradientBoostingClassifer:",scorescvGB*100)
gb.fit(train[final_set], train['hotel_cluster'])
gb_score = gb.score(test[final_set], test['hotel_cluster'])
print("Gradient Boosting Test score:", gb_score*100)


('Gradient Boosting Test score:', 8.2602110729139113)


In [65]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation
gb1 = GradientBoostingClassifier(learning_rate = 0.1, n_estimators = 40, max_depth = 3, min_samples_split = 3, 
                                       random_state=3)
#gb_score = cross_validation.cross_val_score(gb, train[final_set], train['hotel_cluster'], cv=5)

#print("CV Score of GradientBoostingClassifer:",scorescvGB*100)
gb1.fit(train[final_set], train['hotel_cluster'])
gb1_score = gb1.score(test[final_set], test['hotel_cluster'])
print("Gradient Boosting Test score:", gb1_score*100)


('Gradient Boosting Test score:', 9.7867634685193643)


In [66]:
prediction_mGB = gb1.predict(test[final_set])

### Final Prediction:

In [50]:
import itertools
from operator import itemgetter
final_predction = []
for testrow in range(len(test)):
        pred_final = [prediction_m1[testrow],
                                  prediction_m2[testrow], 
                                  prediction_m3[testrow], 
                                  prediction_m4[testrow],
                                  prediction_m5[testrow]]
        final_predction.append(pred_final)  

In [51]:
import ml_metrics as metrics
target = [[l] for l in test['hotel_cluster']]
score = metrics.mapk(target, final_predction, k=5) 
accuracy = score*100
print ("Mean accuracy Precison @5 for Test set is (in %): ",accuracy) 

('Mean accuracy Precison @5 for Test set is (in %): ', 16.500268448846473)


In [59]:
predictedCluster = pd.Series(final_predction)
expectedCluster =  test['hotel_cluster']

In [60]:
final_outcome = pd.concat([predictedCluster, expectedCluster], axis=1)

In [61]:

final_outcome.iloc[1:5, ]                           

,0,hotel_cluster
1,"[86, 81, 91, 91, 86]",82
2,"[48, 68, 9, 9, 47]",99
3,"[13, 63, 74, 63, 67]",9
4,"[2, 81, 91, 91, 2]",2


In [62]:
final_outcome.to_csv("/home/ec2-user/Final/final_predicted.csv")

### Result:Our mean precision accuracy score after ensembling is 16.50%

In [68]:
import itertools
from operator import itemgetter
final_predction1 = []
for testrow in range(len(test)):
        pred_final = [prediction_m1[testrow],
                                  prediction_m2[testrow], 
                                  prediction_mGB[testrow], 
                                  prediction_m4[testrow],
                                  prediction_m5[testrow]]
        final_predction1.append(pred_final)  
import ml_metrics as metrics
target = [[l] for l in test['hotel_cluster']]
GB_score = metrics.mapk(target, final_predction1, k=5) 
Ensemble_accuracy = GB_score*100
print ("Mean accuracy Precison @5 for Test set is using ensemble of gb, rf, erf, knn(in %): ",Ensemble_accuracy)        

('Mean accuracy Precison @5 for Test set is using ensemble of gb, rf, erf, knn(in %): ', 16.753451771361718)


In [69]:
predictedCluster = pd.Series(final_predction1)
expectedCluster =  test['hotel_cluster']
final_outcome1 = pd.concat([predictedCluster, expectedCluster], axis=1)
final_outcome1.to_csv("/home/ec2-user/Final/final_predicted_Set2.csv")

### Conclusion : Using GBM for ensemble, we get 16.75 test accuracy!